In [1]:
# Xóa sạch môi trường hiện tại (không bắt buộc nhưng sạch hơn)
!rm -rf data saved_model *.py

# Copy toàn bộ file .py
!cp /kaggle/input/test-transformer-phomt-700k-dataset-luong/*.py .

# Copy toàn bộ thư mục data
!cp -r /kaggle/input/test-transformer-phomt-700k-dataset-luong/data ./data

# Copy checkpoint
!cp -r /kaggle/input/test-transformer-phomt-700k-dataset-luong/saved_model ./saved_model

In [2]:
!ls saved_model

best_ckpt.tar	 ckpt_epoch2.tar  ckpt_epoch4.tar
ckpt_epoch1.tar  ckpt_epoch3.tar  ckpt_epoch5.tar


In [3]:
import importlib

from main import Manager

manager = Manager(is_train=False, ckpt_name="ckpt_epoch5.tar")

Loading vocabs...
The size of src vocab is 16000 and that of trg vocab is 16000.
Loading Transformer model & Adam optimizer...
Detecting 2 GPUs. Using DataParallel!
Starting from epoch 1
Loading checkpoint...
Resuming training from epoch 5
Setting finished.


In [4]:
import io

src_valid = [line.strip() for line in open("data/src/valid.txt", encoding="utf-8")]
trg_valid = [line.strip() for line in open("data/trg/valid.txt", encoding="utf-8")]

In [5]:
!pip install --upgrade unbabel-comet sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from comet import download_model, load_from_checkpoint
import sacrebleu

# ===================================
# 1. Chạy inference một lần để dùng cho cả BLEU + COMET
# ===================================
srcs = []
preds = []
refs = []

print("Running inference...")

for i, (src, ref) in enumerate(zip(src_valid, trg_valid)):
    pred = manager.inference(src, method="beam")

    srcs.append(src)
    preds.append(pred)
    refs.append(ref)

    if i == 1000:  
        break

# ===================================
# 2. BLEU (SacreBLEU)
# ===================================
bleu = sacrebleu.corpus_bleu(preds, [refs]).score
print("BLEU score:", bleu)

# ===================================
# 3. COMET (SOTA Evaluation)
# ===================================
# Tải và load model COMET (chỉ chạy lần đầu)
model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)

# Tạo input COMET
data = [
    {"src": s, "mt": p, "ref": r}
    for s, p, r in zip(srcs, preds, refs)
]

result = model.predict(data, batch_size=32, gpus=1)
print("COMET score:", result.system_score)

2025-11-29 16:29:42.320211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764433782.518998      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764433782.581281      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Running inference...
Inference starts.
Loading sentencepiece tokenizer...
Preprocessing input sentence...
Encoding input sentence...
Beam search selected.
Input: Reconsider your initial solution and adjust if it is not working .
Result: Đặt lại dung dịch ban đầu và điều chỉnh nếu không hiệu quả .
Inference finished! || Total inference time: 0mins 0secs
Inference starts.
Loading sentencepiece tokenizer...
Preprocessing input sentence...
Encoding input sentence...
Beam search selected.
Input: It can be used with relative effectiveness to get rid of BV .
Result: Nó có thể được sử dụng với hiệu quả tương đối để loại trừ BV .
Inference finished! || Total inference time: 0mins 0secs
Inference starts.
Loading sentencepiece tokenizer...
Preprocessing input sentence...
Encoding input sentence...
Beam search selected.
Input: So he made sure they saw he was well - educated .
Result: Vì vậy ông đảm bảo rằng mình đã được giáo dục tốt .
Inference finished! || Total inference time: 0mins 0secs
Infere

Input: We all talk to one another in this highly visual way without realizing what we 're doing .
Result: Chúng ta đều nói chuyện với nhau theo cách trực quan mà không nhận ra những gì chúng ta đang làm .
Inference finished! || Total inference time: 0mins 0secs
BLEU score: 29.572247352421943


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Predicting DataLoader 0: 100%|██████████| 32/32 [00:22<00:00,  1.45it/s]


COMET score: 0.7931968561716966
